In [3]:
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

### 1. Load BLS raw data

In [4]:
years = ['1990', '2000', '2010', '2020']
NAICS = ['11', '21', '31', '51', '52', '54']
files = []
#NAICS5 = ['51121', '51821', '54151', '54161']

# create list of desired file names, looping through year, BLS folder, and NAICS code
for year in years:
    folder = sorted(os.listdir('BLS_raw/' + year + '.annual.by_industry'))
    for file in folder:
        for code in NAICS:
            if code + ' ' in file[12:15]:
                files.append(file)
            else:
                pass

In [5]:
df_list = []
 
# loop through year and file to append dataframes into a list
for year in years:
    for file in files:
        if year in file:
            temp_df = pd.read_csv('BLS_raw/' + year + '.annual.by_industry/' + file)
            mask_non50 = ((temp_df['area_fips'].str[-3:] == '000') | 
                          (temp_df['area_fips'].str[:1] == 'C') | 
                          (temp_df['area_fips'].str[:2] == 'US') | 
                          (temp_df['area_fips'].str[:1] == '7'))
            temp_df = temp_df[~mask_non50] # apply Kurt's filter within loop
            df_list.append(temp_df)
        else:
            pass

# store dataframes in dictionary
names = []
for x in range(0, len(files)):
    names.append('df' + files[x][12:14]+ '_' + files[x][:4])

d = dict(zip(names, df_list))
d.keys()

dict_keys(['df11_1990', 'df21_1990', 'df51_1990', 'df52_1990', 'df54_1990', 'df11_2000', 'df21_2000', 'df51_2000', 'df52_2000', 'df54_2000', 'df11_2010', 'df21_2010', 'df51_2010', 'df52_2010', 'df54_2010', 'df11_2020', 'df21_2020', 'df51_2020', 'df52_2020', 'df54_2020'])

### 2. Process raw data

In [6]:
# drop undefined counties
for key in d.keys():
    d[key] = d[key][d[key]['area_fips'].str[-3:] != '999']
    print(d[key].shape)

(1111, 43)
(667, 43)
(2197, 43)
(3023, 43)
(2440, 43)
(1156, 43)
(692, 43)
(2301, 43)
(3135, 43)
(2755, 43)
(3293, 43)
(2523, 43)
(5292, 43)
(3975, 43)
(4641, 43)
(3299, 43)
(2492, 43)
(5199, 43)
(3834, 43)
(4733, 43)


In [7]:
# merge public and private sector employment within-county
for key in d.keys():
    d[key] = (d[key].groupby(by=['area_fips', 'area_title'], as_index=False)['annual_avg_emplvl'].sum())
    print(d[key].shape)

(1099, 3)
(667, 3)
(2061, 3)
(2301, 3)
(2003, 3)
(1141, 3)
(692, 3)
(2086, 3)
(2316, 3)
(2097, 3)
(3071, 3)
(2515, 3)
(3081, 3)
(3130, 3)
(3121, 3)
(3091, 3)
(2486, 3)
(3066, 3)
(3125, 3)
(3122, 3)


In [8]:
# load typology (US Census geographical classifications) and regional data
reg = pd.read_csv('typology/regions.csv')
typ = pd.read_csv('typology/typology.csv')

# change county codes to proper fips
typ['fips'] = typ['fips'].astype(str).str.zfill(5)

# merge BLS data with '03, '13, & '20' geographical classifications, retaining all counties from typ file
for key in d.keys():
    d[key] = d[key].merge(typ, how='right', left_on='area_fips', right_on='fips')

# assign regions to BLS data
for key in d.keys():
    for k, v in dict(zip(reg.STATE, reg.REGION)).items():
        d[key].loc[d[key]['State'] == k, ['region']] = v
    d[key].loc[d[key]['area_title'] == 'District of Columbia', 'region'] = 'Mid-Atlantic' # give DC a region
    
# export selected columns
final_cols = ['fips', 'County Title', 'State', 'region',
              'type_census03', 'type_bls13', 'type_census20', 'type_kurt20',
              'annual_avg_emplvl']

for key in d.keys():
    d[key][final_cols].to_csv('my_naics/naics_' + key[2:] + '.csv', index_label=False)

### 3. Calculate employment change columns

In [9]:
# load new NAICS CSVs
codes = ['11', '21', '51', '52', '54']
path = 'my_naics/naics_'

# use big loop to update all previously processed NAICS files
for code in codes:
    df90 = pd.read_csv(path + code + '_1990.csv')
    df00 = pd.read_csv(path + code + '_2000.csv')
    df10 = pd.read_csv(path + code + '_2010.csv')
    df20 = pd.read_csv(path + code + '_2020.csv')
    
    # merge years under NAICS code
    temp1 = df90.merge(df00, how='inner', left_on='fips', right_on='fips', suffixes=['_90', '_00'])
    temp2 = df10.merge(df20, how='inner', left_on='fips', right_on='fips', suffixes=['_10', '_20'])
    df = temp1.merge(temp2, how='inner', left_on='fips', right_on='fips')
    
    # clean column names
    cols = df.columns.tolist()[:9]
    for col in df.columns.tolist()[9:]:
        if col[:-3] == 'annual_avg_emplvl':
            cols.append(col)
        else:
            pass
    df = df[cols]
    df.columns = df.columns.str.replace('_90', '')
    df = df.rename(columns={'annual_avg_emplvl': 'annual_avg_emplvl_90'})
    
    # replace nulls with zeroes 
    empl_cols = df.columns[-4:]
    df[empl_cols] = df[empl_cols].fillna(0)

    # rate of change function
    def rate_chg(df, year1, year2, chg):
        df[chg] = np.where((df[year1]== 0),
                          ((df[year2] - df[year1]) / 1).round(4),
                          ((df[year2] - df[year1]) / df[year1]).round(4))
    
    # define new column namer
    namer = empl_cols.str.split('_')

    # calculate rate of change
    for x in range(0,3):
        rate_chg(df, empl_cols[x], empl_cols[x+1], 'chg_' + namer[x][2] + '_' + namer[x][3] + '_' + namer[x+1][3])
    
    # calculate total rate of change column (1990-2020)
    df['chg_emplvl_90_20'] = np.where((df['annual_avg_emplvl_90']==0),
                                      ((df['annual_avg_emplvl_20'] - df['annual_avg_emplvl_90']) / 1).round(4),
                                      ((df['annual_avg_emplvl_20'] - df['annual_avg_emplvl_90']) / df['annual_avg_emplvl_90']).round(4))
    
    # export file
    print(code, df[df['County Title'].notna()].shape)
    df[df['County Title'].notna()].to_csv('my_naics_chg/naics_' + code + '.csv', index_label=False)

11 (3169, 16)
21 (3169, 16)
51 (3169, 16)
52 (3169, 16)
54 (3169, 16)


In [10]:
df[df['County Title'].isna()]

,fips,County Title,State,region,type_census03,type_bls13,type_census20,type_kurt20,annual_avg_emplvl_90,annual_avg_emplvl_00,annual_avg_emplvl_10,annual_avg_emplvl_20,chg_emplvl_90_00,chg_emplvl_00_10,chg_emplvl_10_20,chg_emplvl_90_20
3169,51933,NaN,NaN,NaN,Rural,Rural,Rural,Metro,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3170,51901,NaN,NaN,NaN,Rural,Rural,Rural,Metro,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3171,51947,NaN,NaN,NaN,Rural,Rural,Rural,Metro,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3172,15901,NaN,NaN,NaN,Rural,Rural,Rural,Metro,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3173,51953,NaN,NaN,NaN,Rural,Rural,Rural,Metro,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3259,72133,NaN,NaN,NaN,Rural,Rural,Micro,Rural,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3260,72055,NaN,NaN,NaN,Rural,Rural,Metro,Rural,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3261,72059,NaN,NaN,NaN,Rural,Rural,Metro,Rural,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3262,72111,NaN,NaN,NaN,Rural,Rural,Metro,Rural,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# check on fips 15005 Kalawao County, so either ignore or treat as incl. in Maui
for key in d.keys():
    print(len(d[key][d[key]['fips'] == '15005']))

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


### 4. Incorporate spatial data

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

# load US counties SHP
gdf_full = gpd.read_file('GIS Data/usa_census_counties_2018_20m/')
gdf_full.set_index('GEOID', inplace=True)

# drop non-continental columns (AK, HI, & PR)
mask_non_continental = ((gdf_full['STATEFP'] == '02') | (gdf_full['STATEFP'] == '15') | (gdf_full['STATEFP'] == '72'))
gdf_continental = gdf_full[~mask_non_continental]

In [ ]:
# load NAICS files with change columns
codes = ['11', '21', '51', '52', '54']
path = 'my_naics_chg/naics_'

for code in codes:
    df = pd.read_csv(path + code + '.csv')
    df['fips'] = df['fips'].astype(str).str.zfill(5)
    df = df.set_index('fips')
    
    # merge with continental gdf
    gdf = gdf_continental.merge(df, how='left', left_index=True, right_index=True)
    
    # export to shapefile
    gdf.to_file('SHPs/NAICS', driver ='ESRI Shapefile')
    
    # SPATIAL ANALYSIS
    for col in gdf.columns[20:]:
        ax = gdf.plot(column=col, cmap='RdYlGn',
                      edgecolor='lightgrey', linewidth=0.1,
                      legend=True, legend_kwds={'shrink': 0.6},
                      vmax=1, vmin=-1,
                      figsize=(15,10),
                      missing_kwds={'color': 'white', 'hatch': 'XXX',
                                    'edgecolor': 'lightgrey', 'linewidth' : 0.2,
                                    'label': 'Null or No Data'})
    
        title = 'NAICS ' + code + ' Industry Employment Dynamics ' + col[11:13] + '-' + col[14:]
        ax.set_title(title, fontsize = 13)
        ax.axis("off")
    
        # save figure
        ax.get_figure().savefig('maps/' + title, dpi=600, bbox_inches="tight")

In [ ]:
# create separate code book for cluster analysis